<h1 id="tocheading">Spring 2018 NLP Class Project: Neural Machine Translation</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import spacy
import pdb
import os
from underthesea import word_tokenize
import jieba
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# ! pip install spacy && python -m spacy download en

## Part 0: Project Overview

The goal of this project is to build a neural machine translation system and experience how recent advances have made their way. Each team will build the following sequence of neural translation systems for two language pairs, __Vietnamese (Vi)→English (En)__ and __Chinese (Zh)→En__ (prepared corpora is be provided):

1. Recurrent neural network based encoder-decoder without attention
2. Recurrent neural network based encoder-decoder with attention
2. Replace the recurrent encoder with either convolutional or self-attention based encoder.
4. [Optional] Build either or both fully self-attention translation system or/and multilingual translation system.

## Part 1: Data Upload & Preprocessing

In [215]:
# start of sentence
SOS_token = 1
# end of sentence
EOS_token = 3

## 2 = unk

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"<PAD>",1: "<SOS>", 2:"<UNK>", 3: "<EOS>"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [216]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    """About "NFC" and "NFD": 
    
    For each character, there are two normal forms: normal form C 
    and normal form D. Normal form D (NFD) is also known as canonical 
    decomposition, and translates each character into its decomposed form. 
    Normal form C (NFC) first applies a canonical decomposition, then composes 
    pre-combined characters again.
    
    About unicodedata.category: 
    
    Returns the general category assigned to the Unicode character 
    unichr as string."""
    
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Trim
def normalizeString(s):
    s = unicodeToAscii(s.strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [217]:
def readLangs(lang1, lang2, reverse=False,
             dataset="train"):
    
    """Takes as input;
    - lang1, lang2: either (vi, en) or (zh, en)
    - dataset: one of ("train","dev","test")"""
    print("Reading lines...")
    eos = [".","?","!","\n"]
    # Read the pretokenized lang1 file and split into lines
    lang1_lines = open("../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-%s-%s-processed/%s.tok.%s" % (lang1, lang2, dataset, lang1), encoding="utf-8").\
        read().strip().split("\n")
    # Read the lang2 file and split into lines
    lang2_lines = open("../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-%s-%s-processed/%s.tok.%s" % (lang1, lang2, dataset, lang2), encoding="utf-8").\
        read().strip().split("\n")
    
    # create sentence pairs (lists of length 2 that consist of string pairs)
    # e.g. ["And we &apos;re going to tell you some stories from the sea here in video .",
    #       "我们 将 用 一些 影片 来讲 讲述 一些 深海 海里 的 故事  "]
    # check if there are the same number of sentences in each set
    assert len(lang1_lines) == len(lang2_lines), "Two languages must have the same number of sentences. "+ str(len(lang1_lines)) + " sentences were passed for " + str(lang1) + "." + str(len(lang2_lines)) + " sentences were passed for " + str(lang2)+"."
    # normalize if not Chinese, Chinese normalization is already handeled
    if lang1 == "zh":
        lang1_lines = lang1_lines
    else:
        lang1_lines = [normalizeString(s) for s in lang1_lines]
    lang2_lines = [normalizeString(s) for s in lang2_lines]
    # construct pairs
    pair_ran = range(len(lang1_lines))
    pairs = [[lang1_lines[i]] + [lang2_lines[i]] for i in pair_ran]
    
#     # Split every line into pairs and normalize
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [218]:
def prepareData(lang1, lang2, reverse=False, dataset="train"):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse, dataset=dataset)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

# example
input_lang, output_lang, pairs = prepareData('vi', 'en', False, dataset="train")
print(random.choice(pairs))

Reading lines...
Read 133317 sentence pairs
Trimmed to 133317 sentence pairs
Counting words...
Counted words:
vi 16144
en 47568
['nhung toi cho rang tren tat ca mot buc tranh tuoi ep ve the gioi nay', 'But I want to argue above all a big picture positive for this world .']


In [219]:
input_lang, output_lang, pairs = prepareData('zh', 'en', False, dataset="train")
print(random.choice(pairs))

Reading lines...
Read 213376 sentence pairs
Trimmed to 213376 sentence pairs
Counting words...
Counted words:
zh 88917
en 59329
['大量 的 样本 是 必须 的 ', 'A very large sample is needed .']


### 1.1 Vietnamese to English

In [220]:
# # Please find the original tokenizing code provided by Elman Mansimov in the following link:
# # https://github.com/derincen/neural-machine-translation/tree/master/data/tokens_and_preprocessing_em/preprocess_translation

# def tokenize_vi(f_names, f_out_names):
#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             tok_lines.write(word_tokenize(sentence, format="text") + '\n')
#         tok_lines.close()

# def tokenize_en(f_names, f_out_names):
#     tokenizer = spacy.load('en_core_web_sm')

#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             # replaced tokenizer(sentence) with str(tokenizer(sentence)) to avoid 
#             # type error while joining
#             tok_lines.write(' '.join(str(tokenizer(sentence))) + '\n')
#         tok_lines.close()


# if __name__ == "__main__":
#     root = '../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-vi-en-processed/'
#     tokenize_vi([os.path.join(root, 'train.vi'), os.path.join(root, 'dev.vi'), 
#                  os.path.join(root, 'test.vi')],\
#                [os.path.join(root, 'train.tok.vi'), os.path.join(root, 'dev.tok.vi'), 
#                 os.path.join(root, 'test.tok.vi')])

#     tokenize_en([os.path.join(root, 'train.en'), os.path.join(root, 'dev.en'), 
#                  os.path.join(root, 'test.en')],\
#                 [os.path.join(root, 'train.tok.en'), os.path.join(root, 'dev.tok.en'), 
#                  os.path.join(root, 'test.tok.en')])


In [221]:
# Format: languagepair_language_dataset
# Train 
vien_vi_train, vien_en_train, vi_en_train_pairs = prepareData('vi', 'en', False, dataset="train")
# Dev 
vien_vi_dev, vien_en_dev, vi_en_dev_pairs = prepareData('vi', 'en', False, dataset="dev")
# Test
vien_vi_test, vien_en_test, vi_en_test_pairs = prepareData('vi', 'en', False, dataset="test")

Reading lines...
Read 133317 sentence pairs
Trimmed to 133317 sentence pairs
Counting words...
Counted words:
vi 16144
en 47568
Reading lines...
Read 1268 sentence pairs
Trimmed to 1268 sentence pairs
Counting words...
Counted words:
vi 1370
en 3816
Reading lines...
Read 1553 sentence pairs
Trimmed to 1553 sentence pairs
Counting words...
Counted words:
vi 1325
en 3619


### 1.2 Chinese to English

In [222]:
# # Please find the original tokenizing code provided by Elman Mansimov in the following link:
# # https://github.com/derincen/neural-machine-translation/tree/master/data/tokens_and_preprocessing_em/preprocess_translation

# def tokenize_zh(f_names, f_out_names):
#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             tok_lines.write(' '.join(jieba.cut(sentence, cut_all=True)))
#         tok_lines.close()

# def tokenize_en(f_names, f_out_names):
#     tokenizer = spacy.load('en_core_web_sm')

#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             # replaced tokenizer(sentence) with str(tokenizer(sentence)) to avoid 
#             # type error while joining
#             tok_lines.write(' '.join(str(tokenizer(sentence))) + '\n')
#         tok_lines.close()

# if __name__ == "__main__":
#     root = '../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-zh-en-processed/'
#     tokenize_zh([os.path.join(root, 'dev.zh'), os.path.join(root, 'test.zh'), os.path.join(root, 'train.zh')],\
#                 [os.path.join(root, 'dev.tok.zh'), os.path.join(root, 'test.tok.zh'), os.path.join(root, 'train.tok.zh')])

# #     tokenize_en([os.path.join(root, 'dev.en'), os.path.join(root, 'test.en'), os.path.join(root, 'train.en')],\
# #                [os.path.join(root, 'dev.tok.en'), os.path.join(root, 'test.tok.en'), os.path.join(root, 'train.tok.en')])


In [223]:
# Format: languagepair_language_dataset
# Train 
zhen_zh_train, zhen_en_train, zh_en_train_pairs = prepareData('zh', 'en', False, dataset="train")
# Dev 
zhen_zh_dev, zhen_en_dev, zh_en_dev_pairs = prepareData('zh', 'en', False, dataset="dev")
# Test
zhen_zh_test, zhen_en_test, zh_en_test_pairs = prepareData('zh', 'en', False, dataset="test")

Reading lines...
Read 213376 sentence pairs
Trimmed to 213376 sentence pairs
Counting words...
Counted words:
zh 88917
en 59329
Reading lines...
Read 1261 sentence pairs
Trimmed to 1261 sentence pairs
Counting words...
Counted words:
zh 6132
en 3916
Reading lines...
Read 1397 sentence pairs
Trimmed to 1397 sentence pairs
Counting words...
Counted words:
zh 5214
en 3423


In [224]:
zh_en_train_pairs[3]

['我们 将 用 一些 影片 来讲 讲述 一些 深海 海里 的 故事  ',
 'And we apos re going to tell you some stories from the sea here in video .']

### 1.3: Check Source & Target Vocabs

Since the source and target languages can have very different table lookup layers, it's good practice to have separate vocabularies for each. Thus, we build vocabularies for each language that we will be using. 

In the first class (Lang) of this section, we have already defined vocabularies for all languages. So, there is no need to redefine another function. We chech each vocabulary below.

#### Chinese Vocabulary

In [225]:
print ("The number of words in Chinese training corpus is " + str(zhen_zh_train.n_words))

The number of words in Chinese training corpus is 88917


In [226]:
zhen_zh_train.word2index["格"]

10479

In [227]:
zhen_zh_train.index2word[10479]

'格'

#### Vietnamese Vocabulary

In [228]:
print ("The number of words in Vietnamese training corpus is " + str(vien_vi_train.n_words))

The number of words in Vietnamese training corpus is 16144


In [229]:
vien_vi_train.word2index["Hamburger"]

6752

In [230]:
vien_vi_train.index2word[6752]

'Hamburger'

#### English Vocabulary for Zh-En

In [231]:
print ("The number of words in English training corpus for Zh-En is " + str(zhen_en_train.n_words))

The number of words in English training corpus for Zh-En is 59329


In [232]:
zhen_en_train.word2index["translate"]

1451

In [233]:
zhen_en_train.index2word[1451]

'translate'

#### English Vocabulary for Vi-En

In [234]:
print ("The number of words in English training corpus for Vi-En is " + str(vien_en_train.n_words))

The number of words in English training corpus for Vi-En is 47568


In [235]:
vien_en_train.word2index["machine"]

847

In [236]:
vien_en_train.index2word[847]

'machine'

### 1.4 Prepare Dataloaders

In [239]:
PAD_IDX = 0
SOS_IDX = 1
UNK_IDX = 2
EOS_IDX = 3
# convert token to id in the dataset
def token2index_dataset(paired_tokens, 
                        lang1_token2id_vocab,
                        lang2_token2id_vocab):
    """Takes as input:
    - paired_tokens: a list of sentence pairs that consist of source & target lang sentences.
    - lang1_token2id_vocab: token2index vocabulary for the first language. 
                            Get by method Lang_dataset.word2index
    - lang2_token2id_vocab: token2index vocabulary for the second language. 
                            Get by method Lang_dataset.word2index
                            
    Returns:
    - indices_data_lang_1, indices_data_lang2: A list of lists where each sub-list holds corresponding indices for each
                                               token in the sentence."""
    indices_data_lang_1, indices_data_lang_2 = [], []
    vocabs = [lang1_token2id_vocab, lang2_token2id_vocab]
    
    # lang1
    for t in range(len(paired_tokens)):
        index_list = [SOS_IDX] + [vocabs[0][token] if token in vocabs[0]\
                                    else UNK_IDX for token in paired_tokens[t][0]] + [EOS_IDX]
        indices_data_lang_1.append(index_list)
    # lang2
    for t in range(len(paired_tokens)):
        index_list = [SOS_IDX] + [vocabs[1][token] if token in vocabs[1] \
                                    else UNK_IDX for token in paired_tokens[t][1]] + [EOS_IDX]
        indices_data_lang_2.append(index_list)
        
    return indices_data_lang_1, indices_data_lang_2

# train indices
zhen_zh_train_indices, zhen_en_train_indices = token2index_dataset(zh_en_train_pairs,
                                                                   zhen_zh_train.word2index,
                                                                   zhen_en_train.word2index)

vien_vi_train_indices, vien_en_train_indices = token2index_dataset(vi_en_train_pairs,
                                                                   vien_vi_train.word2index,
                                                                   vien_en_train.word2index)

# dev indices
zhen_zh_dev_indices, zhen_en_dev_indices = token2index_dataset(zh_en_dev_pairs,
                                                               zhen_zh_dev.word2index,
                                                               zhen_en_dev.word2index)

vien_vi_dev_indices, vien_en_dev_indices = token2index_dataset(vi_en_dev_pairs,
                                                               vien_vi_dev.word2index,
                                                               vien_en_dev.word2index)

# test indices
zhen_zh_test_indices, zhen_en_test_indices = token2index_dataset(zh_en_test_pairs,
                                                                 zhen_zh_test.word2index,
                                                                 zhen_en_test.word2index)

vien_vi_test_indices, vien_en_test_indices = token2index_dataset(vi_en_test_pairs,
                                                                 vien_vi_test.word2index,
                                                                 vien_en_test.word2index)

In [240]:
EOS_IDX

3

In [241]:
# check length
# train
print ("Chinese training set length = "+str(len(zhen_zh_train_indices)))
print ("Chinese-English (En) training set length = "+str(len(zhen_en_train_indices)))
print ("\nVietnamese training set length = "+str(len(vien_vi_train_indices)))
print ("Vietnamese-English (En) training set length = "+str(len(vien_en_train_indices)))
# dev
print ("\nChinese dev set length = "+str(len(zhen_zh_dev_indices)))
print ("Chinese-English (En) dev set length = "+str(len(zhen_en_dev_indices)))
print ("\nVietnamese dev set length = "+str(len(vien_vi_dev_indices)))
print ("Vietnamese-English (En) dev set length = "+str(len(vien_en_dev_indices)))
# test
print ("\nChinese test set length = "+str(len(zhen_zh_test_indices)))
print ("Chinese-English (En) test set length = "+str(len(zhen_en_test_indices)))
print ("\nVietnamese test set length = "+str(len(vien_vi_test_indices)))
print ("Vietnamese-English (En) test set length = "+str(len(vien_en_test_indices)))

Chinese training set length = 213376
Chinese-English (En) training set length = 213376

Vietnamese training set length = 133317
Vietnamese-English (En) training set length = 133317

Chinese dev set length = 1261
Chinese-English (En) dev set length = 1261

Vietnamese dev set length = 1268
Vietnamese-English (En) dev set length = 1268

Chinese test set length = 1397
Chinese-English (En) test set length = 1397

Vietnamese test set length = 1553
Vietnamese-English (En) test set length = 1553


#### Dataloader

In [242]:
UNK_IDX

2

In [243]:
from torch.utils.data import Dataset

In [244]:
len(zhen_zh_train.word2index)

88915

In [245]:
## TODO 

MAX_SENTENCE_LENGTH = 300
BATCH_SIZE = 32

# zhen token2index vocabs
zhen_zh_train_token2id = zhen_zh_train.word2index
zhen_en_train_token2id = zhen_en_train.word2index

# vien token2index vocabs
vien_vi_train_token2id = vien_vi_train.word2index
vien_en_train_token2id = vien_en_train.word2index

class TranslationDataset():
    """
    Class that represents a train/dev/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, 
                 data_source, # training indices data of the source language
                 data_target, # training indices data of the target language
                 token2id_source=None, # token2id dict of the source language
                 token2id_target=None  # token2id dict of the target language
                ):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.source_sentences, self.target_sentences =  data_source, data_target
        
        self.token2id_source = token2id_source
        self.token2id_target = token2id_target

    def __len__(self):
        return len(self.source_sentences)

    def __getitem__(self, batch_index):

#         source_word_idx, target_word_idx = [], []
        source_mask, target_mask = [], []
        
        for index in self.source_sentences[batch_index][:MAX_SENTENCE_LENGTH]:
            if index != UNK_IDX:
                source_mask.append(0)
            else:
                source_mask.append(1)
                
        for index in self.target_sentences[batch_index][:MAX_SENTENCE_LENGTH]:
            if index != UNK_IDX:
                target_mask.append(0)
            else:
                target_mask.append(1)
        
        source_indices = self.source_sentences[batch_index][:MAX_SENTENCE_LENGTH]
        target_indices = self.target_sentences[batch_index][:MAX_SENTENCE_LENGTH]
        
        source_list = [source_indices, source_mask, len(source_indices)]
        target_list = [target_indices, target_mask, len(target_indices)]
        
        return source_list + target_list

    
def translation_collate(batch, max_sentence_length):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    source_data, target_data = [], []
    source_mask, target_mask = [], []
    source_lengths, target_lengths = [], []

    for datum in batch:
        source_lengths.append(datum[2])
        target_lengths.append(datum[5])
        
        # PAD
        source_data_padded = np.pad(np.array(datum[0]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[2])),
                                mode="constant", constant_values=0)
        source_data.append(source_data_padded)
        
        source_mask_padded = np.pad(np.array(datum[1]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[2])),
                                mode="constant", constant_values=0)
        source_mask.append(source_mask_padded)
        
        target_data_padded = np.pad(np.array(datum[3]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[5])),
                                mode="constant", constant_values=0)
        target_data.append(target_data_padded)
        
        target_mask_padded = np.pad(np.array(datum[4]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[5])),
                               mode="constant", constant_values=0)
        target_mask.append(target_mask_padded)
        
    ind_dec_order = np.argsort(source_lengths)[::-1]
    source_data = np.array(source_data)[ind_dec_order]
    target_data = np.array(target_data)[ind_dec_order]
    source_mask = np.array(source_mask)[ind_dec_order].reshape(len(batch), -1, 1)
    target_mask = np.array(target_mask)[ind_dec_order].reshape(len(batch), -1, 1)
    source_lengths = np.array(source_lengths)[ind_dec_order]
    target_lengths = np.array(target_lengths)[ind_dec_order]
    
    source_list = [torch.from_numpy(source_data), 
               torch.from_numpy(source_mask).float(), source_lengths]
    target_list = [torch.from_numpy(target_data), 
               torch.from_numpy(target_mask).float(), target_lengths]
        
    return source_list + target_list


zhen_train_dataset = TranslationDataset(zhen_zh_train_indices,
                                       zhen_en_train_indices,
                                       token2id_source=zhen_zh_train_token2id,
                                       token2id_target=zhen_en_train_token2id)

zhen_train_loader = torch.utils.data.DataLoader(dataset=zhen_train_dataset,
                               batch_size=BATCH_SIZE,
                               collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                               shuffle=False)

zhen_dev_dataset = TranslationDataset(zhen_zh_dev_indices,
                                       zhen_en_dev_indices,
                                       token2id_source=zhen_zh_train_token2id,
                                       token2id_target=zhen_en_train_token2id)

zhen_dev_loader = torch.utils.data.DataLoader(dataset=zhen_dev_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)

vien_train_dataset = TranslationDataset(vien_vi_train_indices,
                                       vien_en_train_indices,
                                       token2id_source=vien_vi_train_token2id,
                                       token2id_target=vien_en_train_token2id)

vien_train_loader = torch.utils.data.DataLoader(dataset=vien_train_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)

vien_dev_dataset = TranslationDataset(vien_vi_dev_indices,
                                       vien_en_dev_indices,
                                       token2id_source=vien_vi_train_token2id,
                                       token2id_target=vien_en_train_token2id)

vien_dev_loader = torch.utils.data.DataLoader(dataset=vien_dev_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)




In [246]:
[*vien_dev_loader][0]

[tensor([[ 1,  2, 83,  ...,  0,  0,  0],
         [ 1, 38,  2,  ...,  0,  0,  0],
         [ 1,  2, 83,  ...,  0,  0,  0],
         ...,
         [ 1,  2, 83,  ...,  0,  0,  0],
         [ 1,  2,  2,  ...,  0,  0,  0],
         [ 1,  2, 83,  ...,  0,  0,  0]]), tensor([[[0.],
          [1.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [1.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [1.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [1.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [1.],
          [1.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [1.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]]), array([283, 204, 204, 182, 178, 160, 15

## Part 2: Evaluation Metric

We use BLEU as the evaluation metric. Specifically, we focus on the corpus-level BLEU function. 

The code for BLEU is taken from https://github.com/mjpost/sacreBLEU/blob/master/sacrebleu.py#L1022-L1080

In [247]:
! pip3 install sacrebleu

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [248]:
import sacrebleu

ModuleNotFoundError: No module named 'sacrebleu'

## Part 3: Beam Search Algorithm

In this section, we implement the Beam Search algorithm in Pytorch.

In [249]:
# initialize k-many score lists
# start only with the whole x
# initialize k-many prev y's lists
# choose top-k for y1 from the whole vocab
# choose top-k for the second time step by expanding the first time step
# compute scores by adding log probabilities

In [250]:
beam_size_k = 10

class BeamSearch:
    
    """RECURSE"""
    
    def __init__(self,
                 beam_size=beam_size_k, ## insert num 
                 softmax_out
                ):
        """
        Class that holds beam information, and search & score functions
        - beam_size = beam size
        - softmax_out = the softmax over the vocabulary at time step t, as computed by the RNN decoder,
                        given the source sequence X and the previously decoded y_<t tokens.
        """
        
        self.beam_size = beam_size
        self.softmax_out = softmax_out
        
        # initialize paths
        self.paths = np.empty((self.beam_size))
        
        # initialize the dictionary that will hold the path scores 
        # and update the scores at each time step
        self.path_score_dict = {}
        # we will later use each i < k as a key and populate this
        # dict with scores

        
    def search():
        
        
        
    
    def score(prev_ys = None):
        """- prev_ys = previously decoded tokens (previously generated target language tokens)
        """
        
    

IndentationError: expected an indented block (<ipython-input-250-fada104b1e20>, line 36)

## Part 4: Model

1. Recurrent neural network based encoder-decoder without attention
2. Recurrent neural network based encoder-decoder with attention
2. Replace the recurrent encoder with either convolutional or self-attention based encoder.

In [251]:
torch.L

AttributeError: module 'torch' has no attribute 'L'

In [489]:
# reconstruction loss = binary cross entropy between two (vocab_size x 1) vectors
# used during training, since we can compare the real Y and and the generated Y
# still at each time step of the decoder, we compare up to and including
# the real t-th token and the generated t-th, then optimize

def loss_function(y_hat, y):
    
    """Takes as input;
    - y: correct "log-softmax"(binary vector) that represents the correct t-th token in the target sentence,
                 (vocab_size x 1) vector
    - y_hat: predicted LogSoftmax for the predicted t-th token in the target sentence.
             (vocab_size x 1) vector
    Returns;
    - NLL Loss in training time"""
    y_hat = torch.log(y_hat)
    loss = nn.functional.binary_cross_entropy(y_hat,y)
    
    return loss
    

# generation/inference time - validation loss = BLEU

def compute_BLEU(corpus_hat,corpus):
    ## TODO
    return None


#### Beam Search

In [490]:
MAX_PATH_LENGTH = 400 # make changeable later !!!

class TargetOut:
    def __init__(self,
                 beam_size=5,
                 source_sentence_length=400,
                 time_step=0):
        """
        - beam: the tensor that will be populated with beam_size-many paths in each timestep
        - beam_size: the width of the beam, top k tokens to include in the beam search,
        """
        
        # initialized again for each timestep
        self.beam = torch.empty(beam_size)
        self.beam_size = beam_size
        self.beam_seq = beam_seq
        self.time_step = time_step
        
        self.max_target_length = source_sentence_length*(1.5)
        # path is kept by hold_path
        self.path = torch.empty(beam_size, max_target_length)
    
    def _add_and_score_paths(self, 
             top_k_tokens):
        
        """top_k_tokens: torch.FloatTensor of indices according to logSoftmax 
        (not embeddings - embedding matrix indices or vocab indices)"""
        
        time_step = self.time_step
        self.path[:,time_step] = top_k_tokens
        
        return self
            
    def _score_paths(self,gru_out):
        
        """For each path, computes log(P(Y_i|Y_i-1,..,Y_i-n,X)) + log(P(Y_i-1|Y_i-2,..,Y_i-n,X)) + ...
        -gru_out is a softmax over the vocabulary for each timestep, so 
        we need to take its log to obtain the scores"""
        if self.time_step = 0:
            
        
    
    def _hold_path_score(self):
        
        





SyntaxError: invalid syntax (<ipython-input-490-9eef64826b7c>, line 39)

### 2.1: RNN-based Encoder-Decoder without Attention

In [491]:
batch_size = 32

In [492]:
torch.cat([torch.FloatTensor([2]).view(-1,1),torch.FloatTensor([3]).view(-1,1)],dim=1)

tensor([[2., 3.]])

In [527]:
# RNN-based encoder-decoder without attention

class RNNencoder(nn.Module):
    
    """Takes as input the variable-length input sequence X, 
    maps it to a fixed-length (final hidden) vector.
    
    We normalize (divide by the length) the final hidden vector.
    BUT, we don't do the same thing for prev_ys"""
    
    def __init__(self,
                 vocab_size=len(zhen_zh_train_token2id), # for chinese
                 embedding_size=300,
                 percent_dropout=0.3, 
                 hidden_size=256,
                 num_gru_layers=1):
        
        super(RNNencoder, self).__init__()
        
        self.vocab_size = vocab_size
        self.embed_size = embedding_size
        self.dropout = percent_dropout
        self.embed_source = nn.Embedding(self.vocab_size,
                                         self.embed_size,
                                         padding_idx=0
                                        )

        self.hidden_size = hidden_size
        self.num_layers = num_gru_layers
        
        self.GRU = nn.GRU(self.embed_size, 
                          self.hidden_size, 
                          self.num_layers, 
                          batch_first=True, bidirectional=True)
        
        self.drop_out_function = nn.Dropout(self.dropout)
        
    def init_hidden(self, batch_size):
        hidden = torch.randn(2*self.num_layers, ## 2 for bidirectional
                             batch_size, self.hidden_size).to(device)
        return hidden
    
    def forward(self, 
                source_sentence, 
                source_mask, 
                source_lengths):
        
        """Returns source lengths to feed into the decoder, since we do not want
        the translation length to be above/below a certain treshold*source sentence length."""
        
        sort_original_source = sorted(range(len(source_lengths)), 
                             key=lambda sentence: -source_lengths[sentence])
        unsort_to_original_source = sorted(range(len(source_lengths)), 
                             key=lambda sentence: sort_original_source[sentence])
        
        source_sentence = source_sentence[sort_original_source]
        _source_mask = source_mask[sort_original_source]
        source_lengths = source_lengths[sort_original_source]
        batch_size, seq_len_source = source_sentence.size()
        # init hidden
        self.hidden_source = self.init_hidden(batch_size)
#         print ("self.hidden_source shape = "+str(self.hidden_source.size()))
        # embeddings for source language
        # carry the commented lines to the decoder
        print ("source sentence shape = "+str(source_sentence.size()))
#         print ("Source Sentence 0 = "+str(source_sentence[0]))
        embeds_source = self.embed_source(source_sentence)
        print ("embeds source size = "+str(embeds_source.size()))
        embeds_source = source_mask*embeds_source + (1-_source_mask)*embeds_source.clone().detach()
        
        embeds_source = torch.nn.utils.rnn.pack_padded_sequence(embeds_source, 
                                                                source_lengths, 
                                                                batch_first=True)
        
        gru_out_source, self.hidden_source = self.GRU(embeds_source, self.hidden_source)
        
        # undo packing
        gru_out_source, _ = torch.nn.utils.rnn.pad_packed_sequence(gru_out_source,
                                                                   batch_first=True)
        
        gru_out_source = gru_out_source.view(batch_size, -1, 2, self.hidden_size)
        gru_out_source = torch.mean(gru_out_source, dim=1) ## mean instead of sum for source representation as suggested in the class
        gru_out_source = torch.cat([gru_out_source[:,i,:] for i in range(2)], dim=1)
        gru_out_source = gru_out_source[unsort_to_original_source] ## back to original indices
        
        # normalizing the hidden vectors in the batch by their own length
        source_lengths = source_lengths[unsort_to_original_source]
#         print ("gru_out_source size = "+str(gru_out_source.size()))
#         print ("gru_out_source = "+str(gru_out_source))
#         print ("source_lengths size = "+str(torch.LongTensor(source_lengths).size()))
#         print ("source_lengths = "+str(source_lengths))
        
        source_lengths_to_norm = torch.transpose(torch.mul(torch.ones(self.hidden_size*2, batch_size),
                                           torch.FloatTensor(source_lengths)),0,1)
        
#         print ("source_lengths_to_norm = "+str(source_lengths_to_norm))
        
        gru_out_source = torch.div(gru_out_source, source_lengths_to_norm)
#         print ("gru_out_source size = "+str(gru_out_source.size()))
        return gru_out_source, source_lengths

In [561]:
class RNNdecoder(nn.Module):
    
    """Given the source sentence and the previously decoded target
    tokens, if any, outputs the softmax over vocabulary for the next token's
    representation. 
    
    We input this softmax to the beam search to get the
    beam_size-many tokens that have the highest probability among all the
    tokens in the vocabulary."""
    
    def __init__(self,
                 vocab_size=len(zhen_en_train_token2id), # for chinese-english's english
                 embedding_size=300,
                 percent_dropout=0.3, 
                 hidden_size=256,
                 num_gru_layers=1
                 ):
        
        super(RNNdecoder, self).__init__()
        
        self.vocab_size = vocab_size
        self.embed_size = embedding_size
        self.dropout = percent_dropout

        self.hidden_size = hidden_size
        self.num_layers = num_gru_layers
        
        self.GRU = nn.GRU(self.embed_size, 
                          self.hidden_size, 
                          self.num_layers, 
                          batch_first=True, bidirectional=True)
        
        self.drop_out_function = nn.Dropout(self.dropout)
        
        self.embed_target = nn.Embedding(self.vocab_size,
                                         self.embed_size, padding_idx=0)
        
        self.sigmoid = nn.Sigmoid()
        
        # the prev_ys tensor at time step 0, 
        # before we have generated any y_s
        self.prev_ys = torch.empty(0)
        
    def init_hidden(self, batch_size):
        hidden = torch.randn(2*self.num_layers, ## 2 for bidirectional
                             batch_size, self.hidden_size).to(device)
        return hidden
        
    def forward(self,
                hidden_source, ## gru_out_source from the encoder
                target_sentence_upto_t, # target sentence tokens generated up to time t
                source_lengths):
        
        # input hidden_source
        self.hidden_source = hidden_source
        
        if target_sentence_upto_t.size(0) == 0:
            gru_out_target = torch.empty(0)
            
        else:
            # init hidden
            self.hidden_target = self.init_hidden(batch_size)
            embeds_target = self.embed_target(target_sentence_upto_t)
            print ("embeds_target = "+str(embeds_target))
            print ("embeds_target size = "+str(embeds_target.size()))
            gru_out_target, self.hidden_target = self.GRU(embeds_target, self.hidden_target)

            # undo packing
            gru_out_target, _ = torch.nn.utils.rnn.pad_packed_sequence(gru_out_target,
                                                                       batch_first=True)

            gru_out_target = gru_out_target.view(batch_size, -1, 2, self.hidden_size)
            gru_out_target = torch.sum(gru_out_target, dim=1) # we don't divide here, just sum
            gru_out_target = torch.cat([gru_out_target[:,i,:] for i in range(2)], dim=1)
            gru_out_target = gru_out_target[unsort_to_original_target] ## back to original indices

            gru_out_target = self.sigmoid(gru_out_target)
        
        # moved linear layers to here, otherwise it will be too crowded
        # concatenate hidden and prev source representations
#         print ("gru out target = "+str(gru_out_target))
#         print ("hidden source = "+str(self.hidden_source))
        hidden = torch.cat([self.hidden_source, gru_out_target], dim=1)
#         print ("hidden = "+str(hidden))
        
        return hidden

In [562]:
torch.empty(0)

tensor([])

In [563]:
# concatenation and linearity
class Linear_Layers(nn.Module):
    
    def __init__(self,
                 input_size=512,
                 hidden_size=1028,
                 percent_dropout_linear=0.3,
                 vocab_size=len(zhen_en_train_token2id) # for chinese-english's english
                ):
        
        super(Linear_Layers, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = percent_dropout_linear
        self.vocab_size = vocab_size

        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.vocab_size)
        self.ReLU = nn.ReLU()
        self.drop_out = nn.Dropout(self.dropout)
        self.log_softmax = nn.LogSoftmax()

        self.init_weights()
        
    def init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_normal_(module.weight)
                nn.init.uniform_(module.bias)

    def forward(self, 
                gru_out, # comes concatenated (source + target upto t)
               ):
        
        gru_out = gru_out.view(gru_out.size(0),-1) 
        print ("gru out size = "+str(gru_out.size()))
        gru_out = self.fc1(gru_out)
        gru_out = self.drop_out(self.ReLU(gru_out))
        out = self.fc2(gru_out)
        
        ## log-softmax over vocabulary
        out = self.log_softmax(out)
#         print ("out from generator = "+str(out))
        print ("generator out shape = "+str(out.size()))
        # not getting the max inside the function to get the vocabulary index 
        # since for the beam search we will need to get the top k indices 
        return out

In [564]:
A = torch.FloatTensor([[2,3,4], [4,5,6]])
A[:,0] = torch.FloatTensor([3,3])

In [565]:
int(torch.max(torch.FloatTensor([2,3,4]),0)[0])

4

In [566]:
torch.eye(10)[9]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [567]:
A

tensor([[3., 3., 4.],
        [3., 5., 6.]])

In [568]:
# RNNenc = # encoder model
# RNNdec = # decoder model
# Generator = # concat and softmax over vocab

In [569]:
def train(RNNenc=RNNencoder,
          RNNdec=RNNdecoder,
          generator=Linear_Layers, # Linear_Layers to generate next 1 token
          loader=zhen_train_loader, # automate
          optimizer=None, 
          epoch=None):
    
    RNNenc.train()
    RNNdec.train()
    generator.train()
    total_loss = 0
    
    for batch_idx, (source_sentence, source_mask, source_lengths, 
                    target_sentence, target_mask, target_lengths)\
    in enumerate(DataLoader):
            
        source_sentence, source_mask = source_sentence.to(device), source_mask.to(device),  
        target_sentence, target_mask = target_sentence.to(device), target_mask.to(device),
        
        RNNenc.train()
        RNNdec.train()
        generator.train()
        optimizer.zero_grad()
        
        generated_target = torch.empty(0)
        
        for t in range(target_sentence.size(1)):
            target_sentence_upto_t = target_sentence[:,t]
            encoded_source = RNNenc(source_sentence,
                                    source_mask,
                                    source_lengths)
        
            encoded_source_concat_target = RNNdec(encoded_source[0], # gru_out_source
                                                  target_sentence_upto_t, # take care of this
                                                  encoded_source[1]  # source lengths
                                                 )
        
            out = generator(encoded_source_concat_target) # returns softmax
            
            print ("out from generator size = "+str(out.size()))
            print ("target sentence size = "+str(target_sentence.size()))
            
            generated_target = torch.cat([generated_target, out], dim=1)
            target_token_softmax = torch.eye(59327)[int(torch.max(out,0)[0])]
            loss = loss_function(out[t], target_sentence[:,t])
            print ("Loss = "+str(loss))
            loss.cuda().backward()
            optimizer.step()

            total_loss += loss.item() * len(sentence1) / len(DataLoader.dataset)
        
        if (batch_idx+1) % (len(DataLoader.dataset)//(20*labels.shape[0])) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx+1) * labels.shape[0], len(DataLoader.dataset),
                100. * (batch_idx+1) / len(DataLoader), loss.item()))

    optimizer.zero_grad()
    return total_loss


# def test(RNN, 
#          Linear_Classifier, 
#          DataLoader, 
#          criterion):

#     RNN.eval()
#     Linear_Classifier.eval()
    
#     test_loss = 0
#     label_list = []
#     output_list = []
    
#     with torch.no_grad():
#         for batch_idx, (sentence1, s1_original, sentence1_lengths, 
#                     sentence2, s2_original, sentence2_lengths, labels)\
#                     in enumerate(DataLoader):

#             sentence1, s1_original = sentence1.to(device), s1_original.to(device),  
#             sentence2, s2_original = sentence2.to(device), s2_original.to(device),
#             labels = labels.to(device)
            
#             # Forward
#             output_s1 = RNN(sentence1, 
#                                   s1_original, 
#                                   sentence1_lengths)
#             # Reverse
#             output_s2 = RNN(sentence2, 
#                                   s2_original, 
#                                   sentence2_lengths)
            
#             out = Linear_Classifier(output_s1, output_s2)
        
#             loss = criterion(out, labels)

#             test_loss += loss.item()/len(DataLoader.dataset)

#             output_list.append(out)
#             label_list.append(labels)
            
#             print ("outputs= "+str(torch.cat(output_list, dim=0)))
#             print ("labels= "+str(torch.cat(label_list, dim=0)))
            
#     return test_loss, torch.cat(output_list, dim=0), torch.cat(label_list, dim=0)

# def accuracy(RNN, 
#              Linear_Classifier, 
#              DataLoader, 
#              criterion):
    
#     _, predicted, true_labels = test(RNN = RNN,
#                               Linear_Classifier = Linear_Classifier,
#                               DataLoader = DataLoader,
#                               criterion = criterion)

#     predicted = predicted.max(1)[1]
#     return 100 * predicted.eq(true_labels.data.view_as(predicted)).float().mean().item()

In [570]:
RNNencoder_model = RNNencoder() # encoder model
RNNdecoder_model = RNNdecoder() # decoder model and concatenation
generator_model = Linear_Layers() # logsoftmax over vocab
DataLoader = zhen_train_loader

num_epochs = 3
lr = 1e-4
# batch_

loss_train = []

for epoch in range(num_epochs):
    print ("epoch = "+str(epoch))

    loss = train(RNNencoder_model,
                 RNNdecoder_model,
                 generator_model,
                       loader = DataLoader,
                       optimizer = torch.optim.Adam(list(RNNencoder_model.parameters()) + \
                                                    list(RNNdecoder_model.parameters()) + \
                                                   list(generator_model.parameters()), 
                                                   lr=lr),
                      epoch = epoch)
    
    loss_train.append(loss)
    
    print (loss_train)

epoch = 0
source sentence shape = torch.Size([32, 300])
embeds source size = torch.Size([32, 300, 300])
embeds_target = tensor([[-0.5268, -0.8680,  0.5817,  ..., -1.9101, -0.8338, -1.1607],
        [-0.5268, -0.8680,  0.5817,  ..., -1.9101, -0.8338, -1.1607],
        [-0.5268, -0.8680,  0.5817,  ..., -1.9101, -0.8338, -1.1607],
        ...,
        [-0.5268, -0.8680,  0.5817,  ..., -1.9101, -0.8338, -1.1607],
        [-0.5268, -0.8680,  0.5817,  ..., -1.9101, -0.8338, -1.1607],
        [-0.5268, -0.8680,  0.5817,  ..., -1.9101, -0.8338, -1.1607]],
       grad_fn=<EmbeddingBackward>)
embeds_target size = torch.Size([32, 300])


RuntimeError: input must have 3 dimensions, got 2

### 2.2 RNN-based Encoder-Decoder with Attention

### 2.3 Encoder Replacement with Eonvolutional or Self-attention-based Encoder

### 2.4 Fully self-attention Translation System

### 2.5 Multilingual Translation System